In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen-3/transformers/0.6b-base/1/config.json
/kaggle/input/qwen-3/transformers/0.6b-base/1/merges.txt
/kaggle/input/qwen-3/transformers/0.6b-base/1/README.md
/kaggle/input/qwen-3/transformers/0.6b-base/1/tokenizer.json
/kaggle/input/qwen-3/transformers/0.6b-base/1/vocab.json
/kaggle/input/qwen-3/transformers/0.6b-base/1/tokenizer_config.json
/kaggle/input/qwen-3/transformers/0.6b-base/1/model.safetensors
/kaggle/input/qwen-3/transformers/0.6b-base/1/generation_config.json
/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv


In [2]:
test = pd.read_csv("/kaggle/input/jigsaw-agile-community-rules/test.csv")
test

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...
5,2034,Top 10 Pentakills in August https://youtu.be/Z...,"No Advertising: Spam, referral links, unsolici...",leagueoflegends,code free tyrande --->>> [Imgur](http://i.imgu...,see tits sweet at this point http://sh.or...,its my asreddit https://www.youtube.com/watch?...,click here for more videos[Russian mafia messe...
6,2035,WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...,"No Advertising: Spam, referral links, unsolici...",AskReddit,"[ENJOY 50,000 BONUS POINTS after $4,000 spent ...",We have high quality Canadian Polymer banknote...,i was on the same ios and try this jailbreak m...,We're streaming Pokemon Veitnamese Crystal RIG...
7,2036,[This mixtape is lit FAM!](https://beholdthear...,"No Advertising: Spam, referral links, unsolici...",BlackPeopleTwitter,watch good one hooters there http://https://x5...,bad pussies waiting for you herein http://gour...,[Full HD Movie Online Free](http://www.flickma...,* Computers :http://livematchstreamz.blogspot....
8,2037,must be watch movie https://sites.google.com/s...,"No Advertising: Spam, referral links, unsolici...",movies,see there hooters sweet http://url.t.aylo...,"Sign up with my link and get $1,000 of free pr...",The fact that there are pro and cons to everyt...,Oh that's interesting. You should check out my...
9,2038,Free paypal cards here!! https://www.pointspri...,"No Advertising: Spam, referral links, unsolici...",pics,she will come your home open her legs with an...,Old and young your girl-next-door make sex wi...,"i think the conjuring 2 is not bad movie ,The...","wow, she's so beautyful\n>[Sakura Vietnam](htt..."


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



base_model_path = "/kaggle/input/qwen-3/transformers/0.6b-base/1"
tokenizer = AutoTokenizer.from_pretrained(base_model_path, local_files_only=True)


model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    local_files_only=True,
    offload_folder="/kaggle/temp_offload"
)

model.eval()



2025-09-13 13:59:48.869178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757771989.058086      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757771989.111306      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [4]:
import torch.nn.functional as F

choices = [
    "Yes, this comment violates the rule.",
    "No, this comment does not violate the rule."
]

def get_choice_probs(row):
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert Reddit content moderator. "
                "Your job is to decide whether a comment violates the given subreddit rule. "
                "You will be given:\n"
                "- The subreddit name\n"
                "- The specific rule text\n"
                "- The comment text to evaluate\n"
                "- Two example comments that violate the rule\n"
                "- Two example comments that do not violate the rule\n\n"
                "Compare the comment with the examples and rule carefully. "
                "Respond only with one of the two options:\n"
                f"1. {choices[0]}\n"
                f"2. {choices[1]}"
            )
        },
        {
            "role": "user",
            "content": (
                f"Subreddit: r/{row['subreddit']}\n"
                f"Rule: {row['rule']}\n\n"
                f"Positive example 1 (violates): {row['positive_example_1']}\n"
                f"Positive example 2 (violates): {row['positive_example_2']}\n"
                f"Negative example 1 (does not violate): {row['negative_example_1']}\n"
                f"Negative example 2 (does not violate): {row['negative_example_2']}\n\n"
                f"Comment to evaluate: {row['body']}"
            )
        }
    ]

    prompt_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    probs = []
    with torch.no_grad():
        for choice in choices:
            choice_ids = tokenizer(choice, return_tensors="pt", add_special_tokens=False).to(model.device)
            input_ids = torch.cat([prompt_ids, choice_ids.input_ids], dim=1)
            attn_mask = torch.cat(
                [torch.ones_like(prompt_ids), torch.ones_like(choice_ids.input_ids)],
                dim=1
            )

            outputs = model(input_ids, attention_mask=attn_mask)
            logits = outputs.logits[:, :-1, :]  # skip last token
            labels = input_ids[:, 1:]           # shifted labels

            log_probs = F.log_softmax(logits, dim=-1)
            token_log_probs = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
            choice_log_prob = token_log_probs[:, -choice_ids.input_ids.size(1):].sum().item()
            probs.append(choice_log_prob)

    probs = torch.softmax(torch.tensor(probs), dim=0)
    return { probs[i].item() for i in range(len(choices))}


In [5]:
pred = []
for _, row in test.iterrows():
    pred.append(list(get_choice_probs(row))[0])


In [6]:
submission = pd.DataFrame({
    "row_id":test["row_id"],
    "rule_violation":pred
})
submission.to_csv("submission.csv" , index=False)

In [7]:
submission

,row_id,rule_violation
0,2029,0.434123
1,2030,0.642011
2,2031,0.351612
3,2032,0.515552
4,2033,0.592914
5,2034,0.526699
6,2035,0.306057
7,2036,0.553940
8,2037,0.518349
9,2038,0.596115
